# Proof of concept DWWC matrix computation

In [1]:
import pandas
from neo4j.v1 import GraphDatabase
import hetio.readwrite
import hetio.neo4j
import hetio.pathtools

from hetmech.degree_weight import dwwc
from hetmech.matrix import get_node_to_position

In [2]:
url = 'https://github.com/dhimmel/hetionet/raw/76550e6c93fbe92124edc71725e8c7dd4ca8b1f5/hetnet/json/hetionet-v1.0.json.bz2'
graph = hetio.readwrite.read_graph(url)
metagraph = graph.metagraph

In [3]:
compound = 'DB01156'  # Bupropion
disease = 'DOID:0050742'  # nicotine dependences

damping_exponent = 0.4

# CbGpPWpGaD contains duplicate metanodes, so DWPC is not equivalent to DWPC
metapath = metagraph.metapath_from_abbrev('CbGpPWpGaD')
metapath.get_unicode_str()

'Compound–binds–Gene–participates–Pathway–participates–Gene–associates–Disease'

In [4]:
%%time
CbGpPWpGaD_pc = dwwc(graph, metapath, damping=0).astype(int)
CbGpPWpGaD_dwwc = dwwc(graph, metapath, damping=damping_exponent)

CPU times: user 16.4 s, sys: 514 ms, total: 16.9 s
Wall time: 10.8 s


In [5]:
CbGpPWpGaD_dwwc.shape

(1552, 137)

In [6]:
# Density
CbGpPWpGaD_pc.astype(bool).mean()

0.8040061328918654

In [7]:
# Path count matrix
CbGpPWpGaD_pc

array([[  11,   13,  192, ...,   40,    0,  758],
       [  23,   30,  400, ...,   77,    0, 1929],
       [  11,   13,  192, ...,   40,    0,  758],
       ..., 
       [   0,    0,    0, ...,    0,    0,    0],
       [   5,    7,   97, ...,   22,    0,  352],
       [   0,    3,  114, ...,    0,    0,  156]])

In [8]:
# DWWC matrix
CbGpPWpGaD_dwwc

array([[ 0.00074141,  0.00057174,  0.01138872, ...,  0.00350678,
         0.        ,  0.02075716],
       [ 0.00057175,  0.00088381,  0.01745733, ...,  0.0050468 ,
         0.        ,  0.03172209],
       [ 0.00074141,  0.00057174,  0.01138872, ...,  0.00350678,
         0.        ,  0.02075716],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.0003131 ,  0.00071464,  0.01298882, ...,  0.00495461,
         0.        ,  0.01838852],
       [ 0.        ,  0.00025659,  0.00779677, ...,  0.        ,
         0.        ,  0.00381554]])

In [9]:
compound_to_position = {x.identifier: i for x, i in get_node_to_position(graph, 'Compound').items()}
disease_to_position = {x.identifier: i for x, i in get_node_to_position(graph, 'Disease').items()}
i = compound_to_position[compound]
j = disease_to_position[disease]

In [10]:
# Path count
CbGpPWpGaD_pc[i, j]

152

In [11]:
# degree-weighted walk count
CbGpPWpGaD_dwwc[i, j]

0.038040121429465001

### Cypher DWPC implementation

In [12]:
query = hetio.neo4j.construct_dwpc_query(metapath, property='identifier', unique_nodes=True)
print(query)

MATCH path = (n0:Compound)-[:BINDS_CbG]-(n1)-[:PARTICIPATES_GpPW]-(n2)-[:PARTICIPATES_GpPW]-(n3)-[:ASSOCIATES_DaG]-(n4:Disease)
USING JOIN ON n2
WHERE n0.identifier = { source }
AND n4.identifier = { target }
AND n1 <> n3
WITH
[
size((n0)-[:BINDS_CbG]-()),
size(()-[:BINDS_CbG]-(n1)),
size((n1)-[:PARTICIPATES_GpPW]-()),
size(()-[:PARTICIPATES_GpPW]-(n2)),
size((n2)-[:PARTICIPATES_GpPW]-()),
size(()-[:PARTICIPATES_GpPW]-(n3)),
size((n3)-[:ASSOCIATES_DaG]-()),
size(()-[:ASSOCIATES_DaG]-(n4))
] AS degrees, path
RETURN
count(path) AS PC,
sum(reduce(pdp = 1.0, d in degrees| pdp * d ^ -{ w })) AS DWPC


In [61]:
driver = GraphDatabase.driver("bolt://neo4j.het.io")
params = {
    'source': compound,
    'target': disease,
    'w': damping_exponent,
}
with driver.session() as session:
    result = session.run(query, params)
    result = result.single()
result

<Record PC=142 DWPC=0.03287590886921623>

### hetio DWPC implementation

In [62]:
compound_id = 'Compound', compound
disease_id = 'Disease', disease
paths = hetio.pathtools.paths_between(
    graph, 
    source=graph.node_dict[compound_id],
    target=graph.node_dict[disease_id],
    metapath=metapath,
    duplicates=False,
)

In [63]:
# Path count
len(paths)

142

In [64]:
# DWPC
dwpc_hetio = hetio.pathtools.DWPC(paths, damping_exponent=damping_exponent)

In [65]:
import numpy
import collections
from scipy.sparse import csr_matrix
import scipy.sparse

from hetmech.matrix import metaedge_to_adjacency_matrix
from hetmech.degree_weight import dwwc_step



def dwpc(graph, metapath, damping=1.0):
    """
    Compute the degree-weighted path count (DWPC).
    First checks that no more than one metanode type is repeated.
    """
    print("Calling DWPC")
    dwpc_matrix = None
    # First read through sequence of metanode types
    nodetype_sequence = collections.defaultdict(int)
    for metaedge in metapath:
        # metanodes = metaedge.get_nodes()
        nodetype_sequence[metaedge.source] += 1
        nodetype_sequence[metaedge.target] += 1
    # Identify the repeated node type
    # and check that no more than 1 metanodetype is repeated
    repeated_node = filter(lambda x: nodetype_sequence[x] > 2,
                           nodetype_sequence)
    repeated_node = list(repeated_node)
    if len(repeated_node) > 1:
        print("Input metapath repeats more than one nodetype")
        return dwpc_matrix

    elif len(repeated_node) == 0:
        print("Input metapath has no repeated nodetypes")
        # Now perform multiplications
        for metaedge in metapath:
            adj_mat = metaedge_to_adjacency_matrix(graph, metaedge)
            adj_mat = dwwc_step(adj_mat, damping, damping)
            if dwpc_matrix is None:
                dwpc_matrix = csr_matrix(adj_mat)
            else:
                dwpc_matrix = dwpc_matrix @ adj_mat
    else:
        # Determine start/endpoints for left,loop,right
        print("Input metapath repeats exactly one nodetype")
        print("Repeated node list: {}, type {}".format(repeated_node, repeated_node[0]))
        
        first_appearance = None
        last_appearance = None
        for idx, metaedge in enumerate(metapath):
            if repeated_node[0] == metaedge.source:
                if first_appearance is None:
                    first_appearance = idx
            if repeated_node[0] == metaedge.target:
                last_appearance = idx
        print("metapath has {} nodes, and {} are the repeated ones"
              "".format(len(metapath),[first_appearance, last_appearance]))

        # Handle head
        left_matrix = None
        for idx, metaedge in enumerate(metapath):
            print("\tWorking on {}".format(idx))
            adj_mat = metaedge_to_adjacency_matrix(graph, metaedge)
            adj_mat = dwwc_step(adj_mat, damping, damping)
            adj_mat = csr_matrix(adj_mat)
            # metanodes = metaedge.get_nodes()
            if idx < first_appearance:  # haven't seen repeatednode yet
                print("\t\tleft_matrix {}".format(idx))
                if left_matrix is None:
                    print("\t\tleft_matrix {} initialize; shape adj {}".format(idx, adj_mat.shape))
                    left_matrix = csr_matrix(adj_mat)
                else:
                    print("\t\tleft_matrix {} multiply; shape left {} ; shape adj {}".format(idx, left_matrix.shape, adj_mat.shape))
                    print("\t\tType of left_matrix before = {}".format(type(left_matrix)))
                    left_matrix = left_matrix @ adj_mat
            elif idx <= last_appearance:  # i.e. metanodes[0] = repeatednode
                print("\t\tloop_matrix {}".format(idx))
                if dwpc_matrix is None:
                    print("\t\tloop_matrix {} initialize; shape adj {}".format(idx, adj_mat.shape))
                    dwpc_matrix = csr_matrix(adj_mat)
                else:
                    print("\t\tloop_matrix {} multiply; shape loop {} ; shape adj {}".format(idx, dwpc_matrix.shape, adj_mat.shape))
                    print("\t\tType of loop_matrix before = {}".format(type(dwpc_matrix)))
                    adj_mat = csr_matrix(adj_mat)
                    dwpc_matrix = dwpc_matrix @ adj_mat
                    # if endpoints are same type, subtract diagonal after mult
                    if metaedge.target == repeated_node[0]:
                        print("\tsubtracting diag {} ".format(idx))
                        # np.diag(diagonals[0], offsets[0])
                        # scipy version here
                        print(dwpc_matrix.diagonal().shape)
                        dwpc_matrix -= scipy.sparse.diags( [dwpc_matrix.diagonal()], [0] )
            else:
                print("\t\ttail_matrix {}".format(idx))
                dwpc_matrix = dwpc_matrix @ adj_mat

    return left_matrix @ dwpc_matrix

In [70]:
from hetmech.path_count import dwpc
dwpc_matrix = dwpc(graph, metapath, damping_exponent)

print("dwpc_matrix shape {}".format(dwpc_matrix.shape))
print("dwpc from i to j, as computed here:     {}".format(dwpc_matrix[i,j]))
print("dwpc from i to j, as computed by hetio: {}".format(dwpc_hetio))

Calling DWPC
dwpc_matrix shape (1552, 137)
dwpc from i to j, as computed here:     0.032875908869216215
dwpc from i to j, as computed by hetio: 0.03287590886921623


In [71]:
dwpc_matrix = dwpc(graph, metapath, 0)

print("dwpc_matrix shape {}".format(dwpc_matrix.shape))
print("path count from i to j, as computed here:     {}".format(dwpc_matrix[i,j]))
print("dwpc path count i to j, as computed by hetio: {}".format(len(paths)))

Calling DWPC
dwpc_matrix shape (1552, 137)
path count from i to j, as computed here:     142.0
dwpc path count i to j, as computed by hetio: 142
